In [1]:
import pyspark
import operator
import datetime

sc = pyspark.SparkContext("local", "TaxiNY")
# sc = pyspark.SparkContext.getOrCreate("local", "TaxiNY")

csvFile1 = sc.textFile("../dataset/trip_data_1.csv")

header = csvFile1.first()
print("\n...................................\n")
# print(header)

# Remove the header
csvFile1 = csvFile1.filter(lambda row: row != header)

# print(csvFile1.first())

csvData = csvFile1.map(lambda lines: lines.split(","))
csvData.cache()

dataLength = csvData.count()
print(f"一共有{dataLength}条出行数据\n")

passengerCountRDD = csvData.map(lambda lines: (lines[7], 1))
passengerCount = passengerCountRDD.reduceByKey(operator.add).collect()

# print(sorted(passengerCount))

allPassengerCount = 0
for i in sorted(passengerCount):
    allPassengerCount += int(i[0]) * int(i[1])

# 乘客人数相关
countString = "人数 次数\n"
with open("./file/passengerCount.txt", "w") as f:
    for i in sorted(passengerCount):
        countString = countString + str(i[0]) + " " + str(i[1]) + "\n"
    countString = countString + "乘客平均数量为：" + str(float(allPassengerCount) / dataLength)
    f.write(countString)

# 一个月中细分的每一天运行次数
dateTimeCountRDD = csvData.map(lambda lines: (datetime.datetime.strptime(lines[5].split(" ")[0], "%Y-%m-%d").date(), 1))
dateTimeCount = dateTimeCountRDD.reduceByKey(operator.add).collect()
countString = "日期 出租车运行次数\n"
with open("./file/dateTimeCount.txt", "w") as f:
    for i in sorted(dateTimeCount):
        countString = countString + str(i[0]) + " " + str(i[1]) + "\n"
    f.write(countString)

# 一天中各个时间段出租车运行的次数
hourTimeCountRDD = csvData.map(lambda lines: (datetime.datetime.strptime(lines[5].split(" ")[1], "%H:%M:%S").hour,1))
hourTimeCount = hourTimeCountRDD.reduceByKey(operator.add).collect()
countString = "时间段 出租车运行次数\n"
with open("./file/hourTimeCount.txt", "w") as f:
    for i in sorted(hourTimeCount):
        countString = countString + str(i[0]) + " " + str(i[1]) + "\n"
    f.write(countString)

# 出租车运行的平均时间
tripTimeTotal= csvData.map(lambda lines: int(lines[8])).sum()
print(f"出租车运行的平均时间为：{float(tripTimeTotal / dataLength)}s")


...................................

一共有14776615条出行数据

出租车运行的平均时间为：683.4235930894863s


In [2]:
import math
# 获取乘车时间的分布(按每一分钟划分）
PartitionTime = 60
tripTimePartitionRDD = csvData.map(lambda lines: (math.ceil(float(lines[8]) / PartitionTime) ,1))
tripTimePartition = tripTimePartitionRDD.reduceByKey(operator.add).sortByKey().collect()
countString = "乘车时间范围(分钟) 车次 \n"
with open("./file/tripTimePartition.txt", "w") as f:
    for i in sorted(tripTimePartition):
        countString = countString + str(i[0]) + "-" +str(i[0] + 1) + " " + str(i[1]) + "\n"
    f.write(countString)


In [5]:
tripDistanceTotal = csvData.map(lambda lines: float(lines[9])).sum()
print(f"出租车运行的平均距离为：{float(tripDistanceTotal / dataLength)}KM")

# 乘车距离分布（按每一千米划分）
tripDistancePartitionRDD = csvData.map(lambda lines: (math.ceil(float(lines[9])), 1))
tripDistancePartition = tripDistancePartitionRDD.reduceByKey(operator.add).sortByKey().collect()
countString = "乘车距离(km) 车次 \n"
with open("./file/tripDistancePartition.txt", "w") as f:
    for i in sorted(tripDistancePartition):
        countString = countString + str(i[0]) + "-" +str(i[0] + 1) + " " + str(i[1]) + "\n"
    f.write(countString)

出租车运行的平均距离为：2.770975670679651KM


In [4]:
# 上车位置聚类
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import KMeans
from pyspark.mllib.clustering import GaussianMixture

correctLocationData = csvData.filter(lambda line: -75 < float(line[10]) < -70 and 36 < float(line[11]) < 45 )
pickUpLocation = correctLocationData.map(lambda lines: Vectors.dense([float(lines[10]), float(lines[11])]))
# print(pickUpLocation.collect())

# KmeansCenter = KMeans.train(pickUpLocation, k=5, maxIterations=10)
# # GaussianMixtureCenter = GaussianMixture.train(pickUpLocation, k=3, maxIterations=10)
# countString = "经度 纬度 \n"
# with open("./file/pickUpLocation.txt", "w") as f:
#     for i in KmeansCenter.clusterCenters:
#     # for i in GaussianMixtureCenter.cluster:
#         countString = countString +str(i[0]) + " " + str(i[1]) + "\n"
#         print(countString)
#     f.write(countString)
# # print(GaussianMixtureCenter.prediction)
print("kan sei")

kan sei
